In [89]:
from itertools import combinations
import pandas as pd
import networkx as nx
import ast
import networkx as nx; 
from networkx.readwrite import json_graph;
import numpy as np
import spacy
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
import time
nlp = spacy.load('en_core_web_sm') #you can use other methods

In [90]:
df = pd.read_csv('../data/processed/headlines_cl_sent.csv')
freq = pd.read_csv('../data/processed/words_freq.csv')

In [91]:
df.shape

(47554, 19)

In [92]:
df.drop_duplicates(subset = ['clean_hl_words'], inplace = True)
df.reset_index(drop = True, inplace = True)

In [10]:
# df

In [93]:
for idx in nlp(' '.join(word for word in ast.literal_eval(df['clean_hl_words'][100]))):
    print(idx)

bloody
phone
thief
m
move


In [133]:
%%time
# exclude nouns from word list
# excluded_tags = {"NOUN", "VERB", "ADJ", "ADV", "ADP", "PROPN"}
included_tags = {"VERB", "ADJ", "ADV"}

df["clean_h1_verbs_adjs"] = df['clean_hl_words'].apply(lambda x: [str(idx) for idx in nlp(' '.join(word for word in ast.literal_eval(x))) if idx.pos_ in included_tags])

Wall time: 5min 17s


In [88]:
# porter = PorterStemmer()
# lancaster=LancasterStemmer()

# print("{0:20}{1:20}{2:20}".format("Word","Porter Stemmer","lancaster Stemmer"))
# for word in female_bias_words:
#     print("{0:20}{1:20}{2:20}".format(word,porter.stem(word),lancaster.stem(word)))

In [134]:
# sources
# http://gender-decoder.katmatfield.com/about
# http://gender-decoder.katmatfield.com/static/documents/Gaucher-Friesen-Kay-JPSP-Gendered-Wording-in-Job-ads.pdf
# https://github.com/DanielleSucher/Jailbreak-the-Patriarchy

# this list of words is used to check for female-gendered language
female_words=set(['actress',  'actresses',  'airwomen',  'alderwoman',  'alderwomen',  'anchorwoman',  'anchorwomen',  
                  'archduchess',  'archduchesses',  'assemblywoman',  'assemblywomen',  'aunt',  'aunts',  'baroness',  
                  'baronesses',  'baronetess',  'baronetesses',  'bogeywoman',  'bogeywomen',  'bondswoman',  'bondswomen',  
                  'bride',  'businesswoman',  'businesswomen',  'camerawoman',  'camerawomen',  'cavewoman',  'cavewomen',  
                  'chairwoman',  'chairwomen',  'clergywoman',  'clergywomen',  'comtesse',  'congresswoman',  'congresswomen',  
                  'councilwoman',  'councilwomen',  'countess',  'countesses',  'countrywoman',  'countrywomen',  'craftswoman',  
                  'craftswomen',  'damsel',  'daughter',  'daughters',  'deaconess',  'deaconesses',  'diva',  'donna’, ‘housewife',  
                  'doorwoman',  'doorwomen',  'duchess',  'duchesses',  'empress',  'empresses',  'fem',  'female',  'females',  
                  'fiancee',  'firewoman',  'firewomen',  'fisherwoman',  'fisherwomen',  'forewoman',  'forewomen',  'freshwoman',  
                  'freshwomen',  'gal',  'galpal',  'gals',  'garbagewoman',  'garbagewomen',  'gentleman',  'girl',  'girlfriend',  
                  'girlfriends',  'girls',  'goddess',  'goddesshead',  'goddesshood',  'goddessliness',  'goddessly',  'godmother',  
                  'granddaughter',  'grandma',  'grandmas',  'grandmother',  'handywoman',  'handywomen',  'hangwoman',  'hangwomen',  
                  'henchwoman',  'henchwomen',  'her',  'heroine',  'heroines',  'herself',  'housewife',  'journeywoman',  'journeywomen',  
                  'kinswoman',  'kinswomen',  'klanswoman',  'ladies',  'ladiez',  'lady',  'lady-romance',  'ladysplain',  'laydeez',  
                  'laywoman',  'laywomen',  "ma'am",  'madam',  'madwoman',  'madwomen',  'maiden',  'mailwoman',  'mailwomen',  'mama',  
                  'marchioness',  'margravine',  'markswoman',  'markswomen',  'marquise',  'middlewoman',  'middlewomen',  'milkwoman',  
                  'milkwomen',  'miss',  'mistress',  'mom',  'momma',  'mommies',  'mommy',  'moms',  'mother',  'mothersmrs',  'ms',  
                  'mum',  'mummy',  'mums',  'niece',  'nieces',  'noblewoman',  'noblewomen',  'ombudswoman',  'ombudswomen',  'policewoman',  
                  'policewomen',  'postwoman',  'postwomen',  'priestess',  'priestesses',  'princess',  'princesses',  'prostitute',  'queen',  
                  'queens',  'repairwoman',  'repairwomen',  'saleswoman',  'saleswomen',  'sandwoman',  'sandwomen',  'servicewoman',  
                  'servicewomen',  'she',  "she's",  'showwoman',  'showwomen',  'sis',  'sistagrammer',  'sistas',  'sister',  'sisters',  
                  'snowwoman',  'spacewoman',  'spacewomen',  'spokeswoman',  'spokeswomen',  'sportswoman',  'sportswomen',  'stateswoman',  
                  'stateswomen',  'stepmother',  'stepsister',  'superwoman',  'superwomen',  'unwoman',  'viscountess',  'viscountesses',  
                  'waitress',  'watchwoman',  'watchwomen',  'weatherwoman',  'weatherwomen',  'widow',  'widows',  'wife',  'wives',  
                  'woman',  'womanhood',  'womankind',  'women',  "women's",  'workwoman',  'workwomen'])

# this list of words
female_bias_words = set(['adorable',  'affair',  'affection',  'affectionate',  'afraid',  'agree',  'angel',  'baby',  'banshee',  'barren',  
                         'beautiful',  'beauty',  'bikini',  'birth',  'bitch',  'bitchfest',  'bitchy',  'body',  'bossy',  'breast',  'bride',  
                         'bridezilla',  'bubbly',  'care',  'caress',  'caring',  'catfight',  'catty',  'chatty',  'cheat',  'cheer',  'child',  
                         'clotheshorse',  'clucky',  'co-operate',  'cold',  'collab',  'commit',  'commitment',  'committed',  'communal',  
                         'compassion',  'compassionate',  'conscientious',  'considerate',  'cook',  'cooking',  'cooperate',  'cougar',  
                         'cries',  'cry',  'crying',  'dedicated',  'demure',  'depend',  'dependable',  'diligent',  'ditzy',  'divorce',  
                         'domestic',  'drama',  'dramatic',  'dress',  'easy',  'emotion',  'emotiona',  'emotional',  'emotions',  'empath',  
                         'empathetic',  'empathize',  'enthusias',  'family',  'fear',  'feel',  'feeling',  'feisty',  'femaleness',  'feminazi',  
                         'feminine',  'fishwife',  'flaky',  'flatter',  'flatterable',  'flirty',  'frail',  'frigid',  'frumpy',  'gentle',  
                         'girlhood',  'girlier',  'girliest',  'girly',  'gossip',  'gossipy',  'helpful',  'honest',  'hormonal',  'houseproud',  
                         'humourless',  'hysterical',  'ice queen',  'inclusive',  'inter-dependen',  'inter-dependence',  'inter-dependent',  
                         'inter-personal',  'interdependence',  'interdependent',  'interpersonal',  'irrational',  'kid',  'kind',  'kinship',  
                         'ladylike',  'lie',  'lippy',  'loose',  'love',  'lover',  'loyal',  'maid',  'makeup',  'man-eater',  'man-hater',  
                         'marriage',  'married',  'marrigeable',  'marry',  'maternal',  'maternal ',  'menstrual ',  'mistress',  'modest',  
                         'modesty',  'moody',  'mousey',  'mumpreneur',  'mumsy',  'nag',  'naked',  'neurotic',  'new born',  'new-born',  
                         'newborn',  'nurse',  'nurtur',  'nurture',  'nurtures',  'nurturing',  'obedient',  'obey',  'over-sensitive',  
                         'pleasant',  'polite',  'powerless',  'pregnancy',  'pretty',  'prostitute',  'prude',  'quiet',  'relationship',  
                         'respon',  'sassy',  'secretary',  'seduce',  'seductive',  'sensitiv',  'sensitive',  'sex',  'sexual',  'sexually',  
                         'share',  'sharin',  'shop',  'shopping',  'shrew',  'single',  'slut',  'slutty',  'soft',  'spinster',  'submissive',  
                         'supermum',  'support',  'sympath',  'sympathy',  'tactful',  'tease',  'tender',  'together',  'tomboy ',  'trollop',  
                         'trust',  'understand',  'victim',  'virgin',  'vivacious',  'warm',  'weak',  'wedding',  'weight',  'whin',  
                         'whitefemaleness',  'whore',  'womanlier',  'womanliest',  'womanliness',  'womanly',  'yield'])

# words we are doubtful about: "model", "accuse", "teen", "young", "home","question", "date", "good", "hair", "couple", "teenage", "fail", "struggle",

male_words = set(['hero', 'man',  'men', 'his', 'him', 'he', 'husband', 'father', 'male', 'son', 'god',
                  'prince', 'king', 'mr', 'sir', 'brother', 'grandfather', 'uncle', 'nephew', 'master', 'patriarch',
                 'chairman', 'chairmen', "boy","boyfriend", "save", "guy", "dad"])

discrimination_words = set(['race', 'caste', 'casteless', 'black', 'SC', 'ST', 'african american', 'white', 'colour', 
                            'color', "brown", "asian", "native", "racial", "minority", "ethnic", "ethnicity", "indian",
                            'hindu', "muslim", "chinese", "indian"])

male_bias_words = set(["active", "adventurous", "aggression",'aggressive', "ambition", "assert", 'assertive',
                       "athlete", 'athletic', "battle", "champion", "decisive", 'head', 'dominate', 'dominant',
                        "driven", 'confident', 'strong', 'force', 'master', 'superior', 'strength', 'bold', 
                       'ambitious', 'power', 'intelligent', 'greedy', 'hostile', 'uncaring', 'logic', 'logical', 'rational',
                       "fearless",'stubborn', 'independent', 'objective', "charismatic"])

empowerement_words = set(['chairperson', 'leader','leadership',  'chairwoman', 'minister', 'power','powerful', 'authority', 
                          'queen', 'manager', 'success', 'successful', 'successes', 'career', 'job',
                         'CEO','CFO', 'chief', 'officer', 'employment', 'employed', 'millionaire', 
                          'wealth', 'wealthy', 'strong', 'strength', 'courage','achievement', 'achievements', 
                          'achieve', 'goal', 'ambition', 'ambitious', 'passionate','passion', 'badass', 
                          'confident', 'confidence', 'breakthrough', "inspirational", "educated"
                         'inspiring', 'inspiration', 'inspire', 'empower', 'empowered', 'empowerement',
                         'genius', 'expert', 'mastery', 'owner', 'businesswoman', 'intelligent', 'smart', 
                          'clever', 'wise', 'worth', 'role model', 'role-model', 'activist', "pay", "work", 
                          "business", "win", "award", "appoint", "lead", "star", "boss", "dream",'goddess', 
                          "actor", 'queen', "launch", "worker", "lawyer", "education", "director", "protester", 
                          "protest", "governor", "survive", "stallion", "doctor", "voice", "perfect", "author",
                          "mayor", "founder", "abortion", "rise", "1st", "winner", "artist", "graduate", 
                          "employee", "earning", "survivor", "scientist", "equality", "equal", "deputy", 
                          "entrepreneur", "survive", "parent"])

politics_words = set(["trump", "biden", "kamala", "harris", "joe", "vote", "election", "president", "elect", 
                      "state", "government", "obama", "office", "campaign", "melania", "vice", "govt", "donald", 
                      "voter", "congress", "candidate", "political", "breonna", "taylor", "pelosi", "democratic",
                      "politic", "democrats", "republican", "ivanka", "republicans", "hillary", "clinton", "susan", 
                      "collins", "warren", "rep", "rally", "debate", "senate", "washington", "speech", "presidential",
                      "boris", "mandela"])

violence_words = set(["find", "allegedly", "fire", "life", 'violent', 'violence', 'crime', 'rape','rapist', 'raped', 'murder','kill', 'killed','killer',
                     'murdered', 'murderer', 'attack', 'alleged', 'criminal', 'stab', 'knife', 'gun', 'guns', 'knives',
                     'blood', 'bloodshed', 'court', 'rage', 'outrage', 'rob', 'steal', 'robber', 'stealer', 'beater', "beaten",
                     'domestic violence', 'aggression', 'aggressor', 'war', 'battle', 'abduction', 'assault', 'assaulted',
                     'drug', 'abuse', 'child abuse', 'prison', 'fraud', 'human traffic', 'homicide', 'organised crime',
                     'organized crime', 'genocide', 'fight', 'manslaughter', 'terrorist', 'weapon', 'smuggl', 'shoplift',
                     'vandalism', 'crime', 'theft', 'penalty', 'prison sentence', 'detained', 'guilty', 'trial',
                     'defense', 'defend', 'armed', 'jail', 'illegal', 'accomplice',
                     'alcohol', 'allegation', 'arson', 'bail', 'battery', 'dead', 'death', 'deadly', 'corrupt', 'killer', 
                     'sex crime', 'wanted', "arrest", "police", "die", "charge", "suspect", "shoot", "sentence", "cop", "hit", 
                     "break", "beat", "judge", "kidnap", "law", "corruption", "gang", "suicide", "critically injured", 'harassment',
                     "run", "crash", "hospital", 'security', "report", "risk", "fall", "burn", "escape", "threaten", "slam",
                     "gangrape", "harass", "brutally", "drown", "justice", "hate", "racist", "allege", "lawsuit", "injure", "racism", 
                      "thug", "suffer", "injury", "horror", "killing", "robbery", "plead", "wound", "kidnapping", "convict", "shooting"])

female_bias = female_words.union(female_bias_words)
male_bias = male_words.union(male_bias_words)

In [98]:
# all_hl_words =[]
# all_marked_words = {}

In [9]:
# all_marked_words =  female_words.union(female_bias_words).union(male_words).union(discrimination_words).union(male_bias_words).union(empowerement_words).union(violence_words)
# #all_marked_words = ast.literal_eval(all_marked_words)
# #list(female_bias)
# all_marked_words = list(all_marked_words)

In [46]:
# def addedges(list):
#     edges = []
#     for i in combinations(list,2):
#         edges.append(i)
#     G.add_edges_from(edges) 
#     print('added edge')

In [120]:
# def usefulwordassociations(list):
#     new_list = [x for x in list if x in all_marked_words]
#     return new_list

In [136]:
def clean_hl_words(all_hl_words):
    for index,value in enumerate(all_hl_words):
        if len(value) == 1:
            all_hl_words.remove(value)

    for index,value in enumerate(all_hl_words):
        if value.isalpha() == False:
            all_hl_words.remove(value)
    return all_hl_words       

In [121]:
# def cleanwordassociations(list):
#     new_list = [x for x in list if x.isalpha == False]
#     new_list = [x for x in new_list if len(x)>1]
#     return new_list

In [54]:
#G = nx.Graph()

In [135]:
def createfullnetwork(df, column="clean_h1_verbs_adjs"):
    G = nx.MultiGraph()
    all_hl_words = []
    for i in range(len(df)):
        word_associations = []
        all_hl_words += clean_hl_words(df[column][i])
        
        #all_hl_words = clean_hl_words(all_hl_words)
        #word_associations = cleanwordassociations(ast.literal_eval(df['clean_hl_words'][i]))
        
        word_associations = df[column][i]
        
        #print(type(word_associations))
        #addedges(word_associations)
        
        edges = []
        for i in combinations(word_associations,2):
            edges.append(i)
        G.add_edges_from(edges) 
        
    #print('added edge')
    G.add_nodes_from(all_hl_words)
    return G

In [137]:
def createWeightedGraph(g_test):
    G = nx.Graph()
    for u,v,data in g_test.edges(data=True):
        w = data['weight'] if 'weight' in data else 1.0
        if G.has_edge(u,v):
            G[u][v]['weight'] += int(w)
        else:
            G.add_edge(u, v, weight=w)
    return G

In [138]:
def getNodeFreq(g, freq):
    nx.set_node_attributes(g, 0, "frequency")
    nx.set_node_attributes(g, 0, "perc_freq")
    for u in g.nodes(data=True):
        node = u[0]
        if len(freq[freq['word']== str(u[0])])>0:
            g.nodes[str(node)]['frequency'] = int(freq[freq['word']== str(u[0])]['frequency'].values[0])
            g.nodes[str(node)]['perc_freq'] = int(freq[freq['word']== str(u[0])]['perc_freq'].values[0]*100000)
    return g

In [139]:
def get_node_themes(g):
    nx.set_node_attributes(g, "", "theme")
    for u in g.nodes(data=True):
        node = u[0]
        if node in female_bias:
             g.nodes[str(node)]['theme'] = "female_bias"
        elif node in male_bias:
             g.nodes[str(node)]['theme'] = "male_bias"
        elif node in empowerement_words:
             g.nodes[str(node)]['theme'] = "empowerment"
        elif node in violence_words:
             g.nodes[str(node)]['theme'] = "violence"
        elif node in politics_words:
             g.nodes[str(node)]['theme'] = "politics"
        elif node in discrimination_words:
             g.nodes[str(node)]['theme'] = "race"
        else:
            g.nodes[str(node)]['theme'] = "other"
            
    return g

In [140]:
def get_edge_themes(g):
    nx.set_edge_attributes(g, "", "theme")
    for u,v,data in g.edges(data=True):
    #     print(data['theme'])
    #     print(g[u][v]['weight'])
        if u in female_bias:
             g[u][v]['theme'] = "female_bias"
        elif u in male_bias:
             g[u][v]['theme'] = "male_bias"
        elif u in empowerement_words:
             g[u][v]['theme'] = "empowerment"
        elif u in violence_words:
             g[u][v]['theme'] = "violence"
        elif u in politics_words:
             g[u][v]['theme'] = "politics"
        elif u in discrimination_words:
             g[u][v]['theme'] = "race"
        else:
            g[u][v]['theme'] = "other"
    return g

In [165]:
def getFinalNetwork(df, freq, edge_filter=2, node_filter=2):
    # create network
    g = createfullnetwork(df)
    # get edge weights
    g = createWeightedGraph(g)
    # get node themes
    g = get_node_themes(g)
    # get edge themes
    g = get_edge_themes(g)
    # get node frequency
    g = getNodeFreq(g, freq)
    print(len(g.nodes), "nodes", "and", len(g.edges), "edges before reducing") 
    # remove irrelevant noise
    ## add irrelevant words here
    words = ['say', 'new', 'time', "south", "africa", "india", "england", "united states", "american", "americans", 
             "UK", "get", "video", "tell", "cape", "makesefakese", "not", "no", "ask", "november", "trump", "biden", 
             "pandemic", "theill", "covid"]
    
    
    
    remove_edges = [(u,v) for u, v, data in g.edges(data=True) if data["weight"] < edge_filter]
    g.remove_edges_from(remove_edges)

    deg = g.degree()
    remove_nodes = [node for node,degree in dict(g.degree()).items() if degree < node_filter]
#     remove_nodes = []
#     remove_nodes = [node[0] for node in g.nodes(data=True) if g.nodes[node[0]]["perc_freq"] <= node_filter]
    remove_nodes.extend(words)
    g.remove_nodes_from(remove_nodes)
    print(len(g.nodes), "nodes", "and", len(g.edges), "edges after reducing")     
    
    return g

In [147]:
def get_country_data(freq, df, country):
    country_freqs = freq[freq['country']==country].reset_index()
    country_df = df[df['country']==country].reset_index()
    return country_df, country_freqs

In [146]:
def saveNetwork(g, country, net_type="all"):
    connections = json_graph.node_link_data(g)
    connections_json = json.dumps(connections, indent = 2) 
    with open(f"../data/processed/word_connections_{country}_{net_type}.json", "w") as outfile: 
        outfile.write(connections_json) 

In [166]:
freq = pd.read_csv("../data/processed/countries_freq.csv")
# country = "India" #South Africa, UK, USA, India
# # words = list(freq[freq['country']!=country].word.values)
# country_freqs = freq[freq['country']!=country].reset_index()
# country_df = df[df['country']==country].reset_index()
ef = 2
nf = 2

g_IN = getFinalNetwork(df = get_country_data(freq=freq, df=df, country = "India")[0], freq = get_country_data(freq=freq, df=df, country = "India")[1], edge_filter=ef, node_filter=nf)
# saveNetwork(g_IN, "IN", net_type="verbs_adjs")
g_USA = getFinalNetwork(df = get_country_data(freq=freq, df=df, country = "USA")[0], freq = get_country_data(freq=freq, df=df, country = "USA")[1], edge_filter=ef, node_filter=nf)
# saveNetwork(g_USA, "USA", net_type="verbs_adjs")
g_UK = getFinalNetwork(get_country_data(freq, df, "UK")[0], get_country_data(freq, df, "UK")[1], edge_filter=ef, node_filter=nf)
# saveNetwork(g_UK, "UK", net_type="verbs_adjs")
g_SA = getFinalNetwork(get_country_data(freq, df, "South Africa")[0], get_country_data(freq, df, "South Africa")[1], edge_filter=ef, node_filter=nf)
# saveNetwork(g_SA, "SA", net_type="verbs_adjs")

2543 nodes and 9920 edges before reducing
271 nodes and 644 edges after reducing
3344 nodes and 15308 edges before reducing
353 nodes and 920 edges after reducing
3488 nodes and 14239 edges before reducing
306 nodes and 843 edges after reducing
1762 nodes and 4799 edges before reducing
171 nodes and 313 edges after reducing


In [168]:
saveNetwork(g_IN, "IN", net_type="verbs_adjs")
saveNetwork(g_USA, "USA", net_type="verbs_adjs")
saveNetwork(g_UK, "UK", net_type="verbs_adjs")
saveNetwork(g_SA, "SA", net_type="verbs_adjs")

In [145]:
freq = pd.read_csv("../data/processed/countries_freq.csv")
# country = "India" #South Africa, UK, USA, India
# # words = list(freq[freq['country']!=country].word.values)
# country_freqs = freq[freq['country']!=country].reset_index()
# country_df = df[df['country']==country].reset_index()
g_IN = getFinalNetwork(df = get_country_data(freq=freq, df=df, country = "India")[0], freq = get_country_data(freq=freq, df=df, country = "India")[1], edge_filter=10, node_filter=200)
saveNetwork(g_IN, "IN_small")
g_USA = getFinalNetwork(df = get_country_data(freq=freq, df=df, country = "USA")[0], freq = get_country_data(freq=freq, df=df, country = "USA")[1], edge_filter=10, node_filter=200)
saveNetwork(g_USA, "USA_small")
g_UK = getFinalNetwork(get_country_data(freq, df, "UK")[0], get_country_data(freq, df, "UK")[1], edge_filter=10, node_filter=200)
saveNetwork(g_UK, "UK_small")
g_SA = getFinalNetwork(get_country_data(freq, df, "South Africa")[0], get_country_data(freq, df, "South Africa")[1], edge_filter=10, node_filter=200)
saveNetwork(g_SA, "SA_small")

13181 nodes and 215872 edges before reducing
91 nodes and 175 edges after reducing
15094 nodes and 294534 edges before reducing
92 nodes and 157 edges after reducing
16674 nodes and 282702 edges before reducing
87 nodes and 142 edges after reducing
10476 nodes and 104227 edges before reducing
83 nodes and 83 edges after reducing


In [132]:
grey_nodes_IN = [node[0] for node in g_IN.nodes(data=True) if g_IN.nodes[node[0]]["theme"] == "other"]
grey_nodes_USA = [node[0] for node in g_USA.nodes(data=True) if g_USA.nodes[node[0]]["theme"] == "other"]
grey_nodes_UK = [node[0] for node in g_UK.nodes(data=True) if g_UK.nodes[node[0]]["theme"] == "other"]
grey_nodes_SA = [node[0] for node in g_SA.nodes(data=True) if g_SA.nodes[node[0]]["theme"] == "other"]

['reveal',
 'real',
 'indian',
 'covid',
 'treatment',
 'security',
 'come',
 'claim',
 'study',
 'hand',
 'question',
 'diwali',
 'post',
 'report',
 'hathras',
 'not',
 'kerala',
 'act',
 'state',
 'government',
 'eat',
 'right',
 'food',
 'school',
 'grow',
 'bengaluru',
 'share',
 'tip',
 'great',
 'risk',
 'long',
 'set',
 'way',
 'tell',
 'official',
 'face',
 'day',
 'obama',
 'leave',
 'office',
 'go',
 'troll',
 'online',
 'date',
 'app',
 'give',
 'struggle',
 'like',
 'fall',
 'single',
 'stay',
 'style',
 'room',
 'pregnancy',
 'shilpa',
 'shetty',
 'happy',
 'hour',
 'singh',
 'character',
 'look',
 'rajput',
 'kapoor',
 'hair',
 'health',
 'sell',
 'late',
 'malaika',
 'arora',
 'help',
 'loss',
 'mask',
 'deepika',
 'padukone',
 'makeup',
 'hang',
 'eye',
 'want',
 'fact',
 'taapsee',
 'pannu',
 'film',
 'ape',
 'priyanka',
 'chopra',
 'world',
 'moment',
 'try',
 'fashion',
 'story',
 'talk',
 'special',
 'sonam',
 'dog',
 'drop',
 'dress',
 'week',
 'adorable',
 'keep'

In [120]:
g_SA.nodes['rape']

{'theme': 'violence', 'frequency': 250, 'perc_freq': 946}

In [97]:
freq[freq['word']== "rape"]['perc_freq'].values[0]

0.009463244757362405

In [108]:
0.0005*100000

50.0

In [105]:
freq[freq['country']=="USA"]['perc_freq']*100000

10415    1431.440078
10416    1363.009165
10417    1312.527344
10418    1020.854601
10419     701.136402
            ...     
29855       1.121818
29856       1.121818
29857       1.121818
29858       1.121818
29859       1.121818
Name: perc_freq, Length: 19445, dtype: float64

In [91]:
len([node[0] for node in g_USA.nodes(data=True) if g_USA.nodes[node[0]]["perc_freq"] <= 0.0005])

KeyError: 'perc_freq'

In [ ]:
len([(u,v) for u, v, data in g_IN.edges(data=True) if data["weight"] <= 4])

In [ ]:
remove_edges = [(u,v) for u, v, data in g.edges(data=True) if data["weight"] <= 3]
remove_nodes = [node[0] for node in g.nodes(data=True) if g.nodes[node[0]]["frequency"] <= 5]

In [201]:
# g_SA = getNodeFreq(g_SA, country_freqs)

In [203]:
# g_SA.edges['man', "rape"]

In [202]:
# g_SA.nodes['man']

In [35]:
# len([node[0]  for node in g_SA.nodes(data=True) if g_SA.nodes[node[0]]["frequency"] <= 5])
len([(u,v) for u, v, data in g_IN.edges(data=True) if data["weight"] <= 5])

1331

In [39]:
# remove_edges = [(u,v) for u, v, data in g_IN.edges(data=True) if data["weight"] <= 3]
remove_nodes = [node[0] for node in g_IN.nodes(data=True) if g_IN.nodes[node[0]]["frequency"] <= 10]

In [221]:
g_SA.remove_edges_from(remove_edges)
g_SA.remove_nodes_from(remove_nodes)

In [220]:
len(remove_edges)

209850

In [16]:
len(g_IN.edges)

2257

In [107]:
g = createfullnetwork(df)

34311
1173780


In [197]:
g = createfullnetwork(df)

34311
1173780


In [109]:
g_weighted = createWeightedGraph(g)

In [137]:
freq.head()

,Unnamed: 0,word,frequency
0,0,say,2146
1,1,new,1940
2,2,man,1774
3,3,trump,1573
4,4,black,1235


In [159]:
frequency = []
nx.set_node_attributes(G, frequency, "frequency")

In [ ]:
for u in g_weighted.nodes(data=True):
    node = u[0]
    if len(freq[freq['word']== str(u[0])])>0:
        g_weighted.nodes[str(node)]['frequency'] = int(freq[freq['word']== str(u[0])]['frequency'].values[0])
   # print(u[0])

In [208]:
g_weighted.nodes['die']

{'frequency': 937}

In [111]:
edges_test = nx.to_pandas_edgelist(G)
edges_test[edges_test['weight']>1]

,source,target,weight
0,crown,v,2
6,crown,di,2
7,crown,diana,3
65,le,di,2
66,le,diana,2
...,...,...,...
12603,supply,problem,2
12606,problem,reckon,2
12868,monthly,archives,4
12871,monthly,archive,3


In [17]:
g = createfullnetwork(df)

32584
0


In [75]:
G = createfullnetwork(df_test)

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
70
357


In [89]:
len(g_test.nodes)

14

In [ ]:
Node attributes to add -
1. which dictionary it belongs to
2. frequency of node
3. frequency of edge

In [7]:
connections = json_graph.node_link_data(g_weighted)
connections_json = json.dumps(connections, indent = 2) 

In [2]:
connections

In [96]:
with open("../data/processed/word_connections.json", "w") as outfile: 
    outfile.write(connections_json) 

In [34]:
#df[(df['headline'].str.lower().str.contains('diana')) & (df['headline'].str.lower().str.contains('crown'))].head()['clean_hl_words']

231     ['crown', 'v', 'realt', 'le', 'foto', 'dei', '...
738     ['crown', 'riveler', 'nuovi', 'segreti', 'anch...
2798    ['play', 'princess', 'diana', 'netflix', 'crown']
3885    ['fact', 'crown', 'star', 'emma', 'corrin', 'l...
3970    ['princess', 'diana', 'camilla', 'parker', 'bo...
Name: clean_hl_words, dtype: object

In [33]:
g.number_of_edges('diana', 'crown')

46

In [126]:
len(g_filtered.nodes)

32592

In [112]:
df['useful_word_associations'] = np.empty((len(df), 0)).tolist()

In [19]:
# import time
# time
# def calcfilterednetwork(df):
#     all_hl_words = []
#     all_useful_hl_words = []
#     for i in range(len(df)):
#         all_hl_words += getusefulnodes(ast.literal_eval(df['clean_hl_words'][i]))
#         df['useful_word_associations'].iloc[i] = usefulwordassociations(ast.literal_eval(df['clean_hl_words'][i]))
        
#         if len(df['useful_word_associations'][i])>=1:
#             addedges(df['useful_word_associations'][i])
            
            
#     G.add_nodes_from(all_hl_words)
    
#     print(len(all_hl_words))  
#     return G

In [126]:
#g_filtered = calcfilterednetwork(df)

#### Themes addition

In [73]:
def read_json_file(filename):
    with open(filename) as f:
        js_graph = json.load(f)
    return json_graph.node_link_graph(js_graph)

In [74]:
g = read_json_file("../data/processed/word_connections_4.json")

In [75]:
theme = ""
nx.set_node_attributes(g, theme, "theme")

In [77]:
def get_node_themes(g):
    for u in g.nodes(data=True):
        node = u[0]
        if node in female_bias:
             g.nodes[str(node)]['theme'] = "female_bias"
        elif node in male_bias:
             g.nodes[str(node)]['theme'] = "male_bias"
        elif node in empowerement_words:
             g.nodes[str(node)]['theme'] = "empowerment"
        elif node in violence_words:
             g.nodes[str(node)]['theme'] = "violence"
        elif node in politics_words:
             g.nodes[str(node)]['theme'] = "politics"
        elif node in discrimination_words:
             g.nodes[str(node)]['theme'] = "race"
        else:
            g.nodes[str(node)]['theme'] = "other"
            
    return g

get_node_themes(g)

In [78]:
g.nodes['police']

{'frequency': 954, 'theme': 'violence'}

In [137]:
g.remove_node("say")
g.remove_node("new")
g.remove_node("time")

In [139]:
nx.set_edge_attributes(g, theme, "theme")

In [80]:
def get_edge_themes(g):
    for u,v,data in g.edges(data=True):
    #     print(data['theme'])
    #     print(g[u][v]['weight'])
        if u in female_bias:
             g[u][v]['theme'] = "female_bias"
        elif u in male_bias:
             g[u][v]['theme'] = "male_bias"
        elif u in empowerement_words:
             g[u][v]['theme'] = "empowerment"
        elif u in violence_words:
             g[u][v]['theme'] = "violence"
        elif u in politics_words:
             g[u][v]['theme'] = "politics"
        elif u in discrimination_words:
             g[u][v]['theme'] = "race"
        else:
            g[u][v]['theme'] = "other"
            
get_edge_themes(g)

In [81]:
g.edges["joe", "biden"]

{'weight': 173.0, 'theme': 'politics'}

In [143]:
connections = json_graph.node_link_data(g)
connections_json = json.dumps(connections, indent = 2) 
with open("../data/processed/word_connections_4_themes.json", "w") as outfile: 
    outfile.write(connections_json) 

In [144]:
remove = [(u,v) for u, v, data in g.edges(data=True) if data["weight"] <= 5]

In [145]:
len(remove)

83723

In [146]:
len(g.edges)

93642

In [147]:
g.remove_edges_from(remove)

In [148]:
len(g.edges)

9919

In [149]:
connections = json_graph.node_link_data(g)
connections_json = json.dumps(connections, indent = 2) 

In [150]:
with open("../data/processed/word_connections_4_themes_filtered.json", "w") as outfile: 
    outfile.write(connections_json) 

#### Countries Networks

In [223]:
g = read_json_file("../data/processed/word_connections_4.json")

In [59]:
freq = pd.read_csv("../data/processed/countries_freq.csv")
country = "South Africa"
words = list(freq[freq['country']!=country].word.values)
country_freqs = freq[freq['country']!=country]

In [226]:
len(g.nodes)

2032

In [43]:
# g.remove_nodes_from(words)

In [65]:
# len(g.nodes)

In [61]:
nx.set_node_attributes(g, 0, "frequency")

In [62]:
g.nodes['die']

{'frequency': 0}

In [63]:
# get specific country frequencies
for u in g.nodes(data=True):
    node = u[0]
    if len(country_freqs[country_freqs['word']== str(u[0])])>0:
        g.nodes[str(node)]['frequency'] = int(freq[freq['word']== str(u[0])]['frequency'].values[0])

In [64]:
g.nodes['die']

{'frequency': 143}